In [1]:
import json
import os
import csv
import re
import glob
import pandas as pd
from pathlib import Path
from collections import Counter
from estnltk import Text
from estnltk.converters import json_to_text
from pandas import DataFrame

In [2]:
def load_wordlist(path):
    """Loeb faili, mis sisaldab stoppsõnu (üks sõna reas)."""
    return Path(path).read_text(encoding='utf-8').splitlines()
    
stopid_s = load_wordlist('estonian-stopwords.txt')
stopid_l = load_wordlist('estonian-stopwords-lemmas.txt')

In [3]:
def process_folder(dirpath: str, gender_tag: str):
    """
    Käib läbi kõik *.jsonid dirpathis ja tagastab kolmiku:
      -  total_minutes (float)
      - full_text (str)
      - sections (sõnastikute loend)
    Kus iga sõnastik sektsioonides sisaldab:
      episode_id, section_index, gender, minutes, transcript
    """
    records = []
    texts = []
    total_seconds = 0.0

    for p in Path(dirpath).glob('*.json'):
        data = json.loads(p.read_text(encoding='utf-8'))
        episode_id = p.stem
        for idx, sec in enumerate(data.get('sections', [])):
            turns = sec.get('turns')
            if not turns:
                continue
            start, end = float(sec['start']), float(sec['end'])
            total_seconds += (end - start)

            # terve tekst
            transcript = ' '.join(t['transcript'] for t in turns)
            texts.append(transcript)

            # metaandmed
            records.append({
                'episode_id':    episode_id,
                'section_index': idx,
                'gender':        gender_tag,
                'minutes':       (end - start) / 60.0,
                'transcript':    transcript
            })

    full_text = ' '.join(texts)
    total_minutes = total_seconds / 60.0
    return total_minutes, full_text, records

In [4]:
def loendamine(kone):
    """
    Salvestab kõik sõnad ja lemmad, mis ei ole kirjavahemärgid, loenditesse.
    """
    
    koik_sonad = []
    koik_lemmad = []
    
    tekst = Text(kone).tag_layer('morph_analysis')
    for sone in tekst.morph_analysis:
        lemma = sone.lemma[0].lower()
        soneliik = sone.partofspeech[0]
                        
        if soneliik != 'Z':
            koik_lemmad.append(lemma)
            koik_sonad.append(sone.text.lower())
            
    return koik_sonad, koik_lemmad

In [5]:
m_minutes, m_text,   men_sections   = process_folder('mehed',  'M')
print('Meeste kõnes kokku on', m_minutes, 'minutit.')
f_minutes, f_text,   women_sections = process_folder('naised','F')
print('Naiste kõnes kokku on', f_minutes , 'minutit.')
all_sections = men_sections + women_sections

print()
print('Meeste kõnes kokku on', round(m_minutes/60, 1), 'tundi')
print('Naiste kõnes kokku on', round(f_minutes/60, 1), 'tundi')

all_words_M, all_lemmas_M = loendamine(m_text)
all_words_F, all_lemmas_F = loendamine(f_text)

Meeste kõnes kokku on 2590.5986666666695 minutit.
Naiste kõnes kokku on 1982.9353333333322 minutit.

Meeste kõnes kokku on 43.2 tundi
Naiste kõnes kokku on 33.0 tundi


In [6]:
print(len(all_words_M))
print(len(all_words_F))

350146
287323


In [7]:
def compute_ttr(tokens):
    """
      tokens: loend str - kõik tokenid (sõnad või lemmad)
    tagastab type- token ratio
    """
    n = len(tokens)
    if n == 0:
        return 0.0
    types = len(set(tokens))
    return types / n

In [8]:
# TTR
ttr_lemmas_M = compute_ttr(all_lemmas_M)
ttr_lemmas_F = compute_ttr(all_lemmas_F)

print("=== Type–Token Ratio (lemmad) ===")
print(f"  Men:   {ttr_lemmas_M:.4f}")
print(f"  Women: {ttr_lemmas_F:.4f}\n")

=== Type–Token Ratio (lemmad) ===
  Men:   0.0573
  Women: 0.0467



In [9]:
m_valitud_minutes, m_valitud_text,   men_valitud_sections   = process_folder('mehed_kinnisvara',  'M')
f_valitud_minutes, f_valitud_text,   women_valitud_sections = process_folder('naised_kinnisvara','F')


print()
print('Meeste kõnes kokku on', round(m_valitud_minutes/60, 1), 'tundi')
print('Naiste kõnes kokku on', round(f_valitud_minutes/60, 1), 'tundi')

all_words_M_valitud, all_lemmas_M_valitud = loendamine(m_valitud_text)
all_words_F_valitud, all_lemmas_F_valitud = loendamine(f_valitud_text)

# TTR
ttr_lemmas_M_valitud = compute_ttr(all_lemmas_M_valitud)
ttr_lemmas_F_valitud = compute_ttr(all_lemmas_F_valitud)

print("=== Type–Token Ratio (lemmad) ===")
print(f"  Men:   {ttr_lemmas_M_valitud:.4f}")
print(f"  Women: {ttr_lemmas_F_valitud:.4f}\n")


Meeste kõnes kokku on 0.4 tundi
Naiste kõnes kokku on 0.4 tundi
=== Type–Token Ratio (lemmad) ===
  Men:   0.2612
  Women: 0.1906



In [10]:
m_valitud_minutes, m_valitud_text,   men_valitud_sections   = process_folder('mehed_esoteric',  'M')
f_valitud_minutes, f_valitud_text,   women_valitud_sections = process_folder('naised_esoterc','F')

print()
print('Meeste kõnes kokku on', round(m_valitud_minutes/60, 1), 'tundi')
print('Naiste kõnes kokku on', round(f_valitud_minutes/60, 1), 'tundi')

all_words_M_valitud, all_lemmas_M_valitud = loendamine(m_valitud_text)
all_words_F_valitud, all_lemmas_F_valitud = loendamine(f_valitud_text)

# TTR
ttr_lemmas_M_valitud = compute_ttr(all_lemmas_M_valitud)
ttr_lemmas_F_valitud = compute_ttr(all_lemmas_F_valitud)

print("=== Type–Token Ratio (lemmad) ===")
print(f"  Men:   {ttr_lemmas_M_valitud:.4f}")
print(f"  Women: {ttr_lemmas_F_valitud:.4f}\n")


Meeste kõnes kokku on 0.8 tundi
Naiste kõnes kokku on 0.7 tundi
=== Type–Token Ratio (lemmad) ===
  Men:   0.1676
  Women: 0.1620



In [11]:
def compute_lemma_length(tokens):
    
    n = len(tokens)
    average_len = sum(len(word) for word in tokens) / n
    
    return average_len

In [12]:
len_lemmas_M = compute_lemma_length(all_lemmas_M)
len_lemmas_F = compute_lemma_length(all_lemmas_F)

print("=== Keskmine pikkus (lemmad) ===")
print(f"  Men:   {len_lemmas_M:.4f}")
print(f"  Women: {len_lemmas_F:.4f}\n")

=== Keskmine pikkus (lemmad) ===
  Men:   4.8046
  Women: 4.9251



Sagedusloendid

In [13]:
def freq_list(lemmas, stop_words):
    """
    Loeb leemade sagedusloendi.
    """


    freq_counter = Counter()
    for lemma in lemmas:
        if lemma not in stop_words:
            freq_counter[lemma] += 1

    return freq_counter

In [14]:
def freq_fails(failinimi, loend):
    """
    Salvestab lemmade sagedusloendi faili.
    """
    
    with open(failinimi, 'w', encoding='utf-8', newline='') as out_f:
        writer = csv.writer(out_f, delimiter=';')
        writer.writerow(['lemma', 'raw_count', 'freq_per_1000_lemmas'])
        for lemma, cnt in loend.most_common():
            freq_per_1000_lemmas = (cnt / len(loend)) * 1000
            writer.writerow([lemma, cnt, f"{freq_per_1000_lemmas:.3f}"])

In [15]:
filename_out_M = 'n_grammid/lemmade_sagedus_by_min_M.csv'
filename_out_F = 'n_grammid/lemmade_sagedus_by_min_F.csv'

freq_fails(filename_out_M, freq_list(all_lemmas_M, stopid_l))
freq_fails(filename_out_F, freq_list(all_lemmas_F, stopid_l))

Bigrammid ja trigrammid:

In [16]:
def bi_and_tri_grams(lemmas, stop_lemmas, words, stop_words):
    """
    Loendab lemmasõnade ja algsõnade bigramme ja trigramme 
    lihtsustatud stoppsõnade filtreerimise reeglitega.
    
    Tagastab: (bigram_lem, trigram_lem, bigram_raw, trigram_raw)
    """
    def count_ngrams(tokens, stop_set):
        tokens_lower = [t.lower() for t in tokens]
        N = len(tokens_lower)
        bigrams = Counter()
        trigrams = Counter()
        # bigrammid: loendatakse, kui teine element ei ole stoppsõnades ja mõlemad elemedid on erinevad
        for i in range(N - 1):
            w1, w2 = tokens_lower[i], tokens_lower[i+1]
            if w1 != w2 and w2 not in stop_set:
                bigrams[(w1, w2)] += 1
        # trigrammid: loendatakse, kui kõik kolm on erinevad ja esimene ning kolmas elemendid ei ole stoppsõnades.
        for i in range(N - 2):
            w1, w2, w3 = tokens_lower[i], tokens_lower[i+1], tokens_lower[i+2]
            if len({w1, w2, w3}) == 3 and w1 not in stop_set and w3 not in stop_set:
                trigrams[(w1, w2, w3)] += 1
        return bigrams, trigrams

    stop_lemmas_set = set(stop_lemmas)
    stop_words_set  = set(stop_words)

    # lemmade ja algsõnade kohta n-grammide loendamine
    bigram_lem, trigram_lem = count_ngrams(lemmas, stop_lemmas_set)
    bigram_raw, trigram_raw = count_ngrams(words,  stop_words_set)

    # top 10
    def print_top(cnt: Counter, title: str):
        print(f"Top 10 {title}:")
        for ngram, freq in cnt.most_common(10):
            print(ngram, freq)
        print()

    print_top(bigram_lem,   "bigrams (lemmas)")
    print_top(trigram_lem,  "trigrams (lemmas)")
    print_top(bigram_raw,   "bigrams (raw forms)")
    print_top(trigram_raw,  "trigrams (raw forms)")

    return bigram_lem, trigram_lem, bigram_raw, trigram_raw

In [17]:
print('Mehed:')
print()
bigrammid_lem_M, trigrammid_lem_M, bigrammid_raw_M, trigrammid_raw_M = bi_and_tri_grams(all_lemmas_M, stopid_l, all_words_M, stopid_s)
print()
print('Naised:')
print()
bigrammid_lem_F, trigrammid_lem_F, bigrammid_raw_F, trigrammid_raw_F = bi_and_tri_grams(all_lemmas_F, stopid_l, all_words_F, stopid_s)

Mehed:

Top 10 bigrams (lemmas):
('ei', 'teadma') 854
('mina', 'arvama') 656
('see', 'mõte') 531
('mina', 'ütlema') 346
('see', 'asi') 302
('saama', 'aru') 298
('mina', 'tahtma') 289
('mina', 'arust') 273
('kogu', 'aeg') 236
('mina', 'rääkima') 182

Top 10 trigrams (lemmas):
('eesti', 'puue', 'inimene') 28
('puue', 'inimene', 'koda') 27
('mees', 'ei', 'nutma') 20
('no', 'see', 'mõte') 17
('teadma', 'mis', 'asi') 13
('no', 'mina', 'ütlema') 12
('jutt', 'roheline', 'stuudio') 10
('ütlema', 'et', 'kuulma') 10
('tõde', 'ja', 'õigus') 9
('no', 'mina', 'arvama') 9

Top 10 bigrams (raw forms):
('ei', 'tea') 793
('ma', 'arvan') 597
('selles', 'mõttes') 467
('minu', 'arust') 272
('kogu', 'aeg') 234
('ma', 'ütlen') 145
('see', 'ongi') 127
('väga', 'hea') 122
('ma', 'tean') 122
('saan', 'aru') 118

Top 10 trigrams (raw forms):
('eesti', 'puuetega', 'inimeste') 22
('mehed', 'ei', 'nuta') 18
('no', 'selles', 'mõttes') 17
('jutud', 'rohelises', 'stuudios') 10
('no', 'ma', 'arvan') 9
('puuetega', 'in

In [18]:
def save_ngrams(counter, out_path, sep: str = ';', precision: int = 4):
    """
   Salvestab n-grammid loendurist CSV-formaadis veergudega:
      ngramm, count, freq_per_1000_n-gramms
    """
    total_ngrams = sum(counter.values())
    
    factor = 1000.0 / total_ngrams if total_ngrams else 0.0
    
    p = Path(out_path)
    p.parent.mkdir(parents=True, exist_ok=True)
    with p.open('w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f, delimiter=sep)
        writer.writerow(['ngram', 'count', 'freq_per_1000_n-gramms'])
        fmt = f'{{:.{precision}f}}'
        for ngram, cnt in counter.most_common():
            # Kui ngram on tupel, siis liimi, muidu valatakse str-iks
            text = ' '.join(ngram) if isinstance(ngram, (list, tuple)) else str(ngram)
            writer.writerow([text, cnt, fmt.format(cnt * factor)])

In [19]:
def bi_ja_tri_failid(bigram_lem_fpath,   trigram_lem_fpath,
                    bigram_raw_fpath,   trigram_raw_fpath,
                    bigram_counter_lem, trigram_counter_lem,
                    bigram_counter_raw, trigram_counter_raw):
    """
    Salvestab neli CSV-d: lemmade bigrammid/trigrammid ja algsõnade bigrammid/trigrammid.
    """
    mapping = {
        bigram_lem_fpath:   bigram_counter_lem,
        trigram_lem_fpath:  trigram_counter_lem,
        bigram_raw_fpath:   bigram_counter_raw,
        trigram_raw_fpath:  trigram_counter_raw,
    }
    for path, counter in mapping.items():
        save_ngrams(counter, path)

In [20]:
filename_bigrams_lemma_M = 'n_grammid/bigrams_lemma_per_1000_ngramms_mehed.csv'
filename_trigrams_lemma_M = 'n_grammid/trigrams_lemma_per_1000_ngramms_mehed.csv'
filename_bigrams_raw_M = 'n_grammid/bigrams_raw_per_1000_ngramms_mehed.csv'
filename_trigrams_raw_M = 'n_grammid/trigrams_raw_per_1000_ngramms_mehed.csv'

filename_bigrams_lemma_F = 'n_grammid/bigrams_lemma_per_1000_ngramms_naised.csv'
filename_trigrams_lemma_F = 'n_grammid/trigrams_lemma_per_1000_ngramms_naised.csv'
filename_bigrams_raw_F = 'n_grammid/bigrams_raw_per_1000_ngramms_naised.csv'
filename_trigrams_raw_F = 'n_grammid/trigrams_raw_per_1000_ngramms_naised.csv'

bi_ja_tri_failid(filename_bigrams_lemma_M, filename_trigrams_lemma_M, filename_bigrams_raw_M, filename_trigrams_raw_M, 
                    bigrammid_lem_M, trigrammid_lem_M, bigrammid_raw_M, trigrammid_raw_M)
bi_ja_tri_failid(filename_bigrams_lemma_F, filename_trigrams_lemma_F, filename_bigrams_raw_F, filename_trigrams_raw_F, 
                    bigrammid_lem_F, trigrammid_lem_F, bigrammid_raw_F, trigrammid_raw_F)

Sõnaliigid:

In [21]:
def count_pos(text_str: str,
              skip_lemmas: set[str],
              stop_lemmas: set[str]
             ) -> dict[str, Counter]:
    """
    Loendab tekstis lemmade esinemissagedust sõnaliikide kaupa.

    Kui eelmise token'i lemmavorm kattub käesoleva token'i lemmaga,
    siis käesolevat token'it ei arvestata loendurites.

    Parameters
    ----------
    text_str : str
        Analüüsitav algtekst.
    skip_lemmas : set[str]
        Lemmad (väiketähtedega), mida ignoreeritakse (nt sagedased sõnad).
    stop_lemmas : set[str]
        Abiverbide lemmad (nt 'olema', 'saama' jm), mille puhul loendame
        abiverbide alla.
    
    Returns
    -------
    dict[str, Counter]
        Loendurid sõnaliikide kaupa.
    """
    txt = Text(text_str)
    txt.tag_layer('morph_analysis')

    counters: dict[str, Counter] = {
        'omadussõnad': Counter(),
        'nimisõnad':   Counter(),
        'adverbid':    Counter(),
        'verbid':      Counter(),
        'abiverbid':   Counter(),
        'asesõnad':    Counter(),
        'kaassõnad':   Counter(),
    }

    tag_map = {
        'A': 'omadussõnad',
        'S': 'nimisõnad',
        'D': 'adverbid',
        'P': 'asesõnad',
        'K': 'kaassõnad',
    }

    prev_lemma: str | None = None        # lemmavorm eelmisest tokenist
    for token in txt.morph_analysis:
        lemma = token.lemma[0].lower()
        tag   = token.partofspeech[0]

        # Jäta vahele, kui lemmaga tuleb alati vahele jätta
        if lemma in skip_lemmas:
            prev_lemma = lemma           # uuenda prev_lemma ja jätka
            continue

        # Jäta vahele, kui eelmise token'i lemmaga sama
        if prev_lemma == lemma:
            prev_lemma = lemma
            continue

        # Loenda verbid / abiverbid
        if tag == 'V':
            bucket = 'abiverbid' if lemma in stop_lemmas else 'verbid'
            counters[bucket][lemma] += 1

        # Loenda muud huvipakkuvad sõnaliigid
        elif tag in tag_map:
            counters[tag_map[tag]][lemma] += 1

        # Uuenda eelmise lemmavormi meelespidamine
        prev_lemma = lemma

    return counters

In [22]:
## Lemmad, mis tähistavad eitust
negation = ['ei', 'ära']

# DataFrame
records = []
for sec in all_sections:
    # Sageduste loendamine sõnaliikide kategooriate kaupa transkriptsiooni lõigu kohta
    counts = count_pos(sec['transcript'], negation, stopid_l)
    for pos, counter in counts.items():
        for form, cnt in counter.items():
            if sec['gender'] == 'M':
                freq = (cnt / len(all_words_M)) * 1000 if len(all_words_M) > 0 else 0.0
            else:
                freq = (cnt / len(all_words_F)) * 1000 if len(all_words_F) > 0 else 0.0
            records.append({
                'episode_id': sec['episode_id'].split('_', 1)[0],    # episoodi id
                'section_index': sec['section_index'],# lõigu järjekorranumber episoodis
                'gender': sec['gender'],              # esineja sugu
                'minutes': sec['minutes'],            # lõigu kestus minutites
                'pos': pos,                           # sõnaliik (count_pos võti)
                'feature': form,                      # sõna 
                'raw_count': cnt,                     # esinemiste arv tekstis
                'freq_per_1000_tokens': freq               # 1000 sõnede normaliseeritud sagedus
            })

df_sections_pos = pd.DataFrame(records)

# Salvestame iga POS-i jaoks eraldi CSV-faili
os.makedirs('pos_sections_csv', exist_ok=True) #vaata GitHubis sõnaliigid kausta
for pos in df_sections_pos['pos'].unique():
    df_subset = df_sections_pos[df_sections_pos['pos'] == pos]
    filename = f'pos_sections_csv/{pos}_by_section.csv'
    df_subset.to_csv(filename, index=False)

     # Teade salvestamise kohta
    print(f"Saved {len(df_subset)} lines - {filename}")

Saved 15253 lines - pos_sections_csv/omadussõnad_by_section.csv
Saved 58428 lines - pos_sections_csv/nimisõnad_by_section.csv
Saved 34869 lines - pos_sections_csv/adverbid_by_section.csv
Saved 25712 lines - pos_sections_csv/verbid_by_section.csv
Saved 3402 lines - pos_sections_csv/abiverbid_by_section.csv
Saved 8783 lines - pos_sections_csv/asesõnad_by_section.csv
Saved 5379 lines - pos_sections_csv/kaassõnad_by_section.csv


In [23]:
input_dir = 'pos_sections_csv'  #vaata GitHubis sõnaliigid kausta
output_dir = 'pos_sections_csv/pos_sections_counter'
# Loome väljundkausta, kui seda veel pole
os.makedirs(output_dir, exist_ok=True)

# Töötleme iga CSV-faili sisendkaustas

for filepath in glob.glob(os.path.join(input_dir, '*.csv')):
    df = pd.read_csv(filepath)
    
    grouped = (
        df
        .drop(columns=['feature'])
        .groupby(['episode_id', 'section_index', 'gender', 'minutes', 'pos'], as_index=False)
        .agg({
            'raw_count': 'sum',          
    )
        
    def calc_freq(row):
        cnt = row['raw_count']
        if row['gender'] == 'M':
            return (cnt / all_words_M) * 1000 if all_words_M > 0 else 0.0
        else:
            return (cnt / all_words_F) * 1000 if all_words_F > 0 else 0.0

    grouped['freq_per_1000_tokens'] = grouped.apply(calc_freq, axis=1)
            
    filename = os.path.basename(filepath)
    output_path = os.path.join(output_dir, filename)
    grouped.to_csv(output_path, index=False)
    
     # Teade salvestamise kohta
    print(f'{filename} → {output_path}')

kaassõnad_by_section.csv → pos_sections_csv/pos_sections_counter/kaassõnad_by_section.csv
omadussõnad_by_section.csv → pos_sections_csv/pos_sections_counter/omadussõnad_by_section.csv
abiverbid_by_section.csv → pos_sections_csv/pos_sections_counter/abiverbid_by_section.csv
verbid_by_section.csv → pos_sections_csv/pos_sections_counter/verbid_by_section.csv
adverbid_by_section.csv → pos_sections_csv/pos_sections_counter/adverbid_by_section.csv
nimisõnad_by_section.csv → pos_sections_csv/pos_sections_counter/nimisõnad_by_section.csv
asesõnad_by_section.csv → pos_sections_csv/pos_sections_counter/asesõnad_by_section.csv


In [24]:
eituse_partiklid = ['ei', 'ära']
es_ja_te_pronoomenid = ['mina', 'sina']
tingiv_koneviis = ['ks', 'ksin', 'ksid', 'ksime', 'ksite', 'taks', 'nuksin', 'nuksid', 'nuks', 'nuksime', 'nuksite', 'tuks']
umbisikuline = ['ti', 'takse', 'ta', 'tud', 'taks', 'tuks']
kaskiv_koneviis = ['o', 'ge', 'gem']
kp_markerid = ['arvan', 'usun', 'kardan', 'tundub', 'paistab', 'näib', 'loodan', 'ütleme']
verbid_partiklitena = ['ootama', 'vaatama', 'kuulama', 'kuulma']

intensiivistajad_loendist = []
with open('intensiivistajad.txt', 'r', encoding='utf-8') as f:
    for line in f:
        intensiivistajad_loendist.append(line.strip())

In [25]:
# Initsialiseerime tühjad loendid erinevate tunnuste jaoks
eitused = []                               # eitused (näiteks 'ei' + verb)
esimese_ja_teise_isik = []                # 1. ja 2. isiku pronoomenid
intensiivistajad_omadussonadega = []      # intensiivistajad + omadussõnad
intensiivistajad_verbidega = []           # intensiivistajad + verbid
intensiivistajad_adverbidega = []         # intensiivistajad + adverbid
tingiv_koneviis_loend = []                # tingiv kõneviis
umbisikuline_loend = []                   # umbisikuline vorm
kaskiv_koneviis_loend = []                # käsiv kõneviis
kp_markerid_loend = []                    # kõnepartiklid markeritena
verbid_partiklitena_loend = []           # verbid, mis on partiklitena
partiklid = []                            # kõik leitud partiklid
intensiivistajad = []
deminutiivid = []

for sec in all_sections:
    plok = sec['transcript']

    plok_nltk = Text(plok).tag_layer('morph_analysis')
    for lause in plok_nltk.sentences:      
        for i, sone in enumerate(lause):
            episodi_id = sec['episode_id'].split('_', 1)[0]
            if sone.lemma[0].endswith('kene') and sone.lemma[0] not in ['kene', 'skene']:
                deminutiivid.append({
                    'episode_id': episodi_id,
                    'section_index': sec['section_index'],
                    'gender': sec['gender'],
                    'minutes': sec['minutes'],
                    'sõna': sone.lemma[0],
                    'lause': " ".join(tok.text for tok in lause)
                })
            # 1) Eitused: eituse partikkel + järgneb verb
            if sone.lemma[0] in eituse_partiklid and i+1 < len(lause) and lause[i+1].partofspeech[0] == 'V':
                eitused.append({
                    'episode_id': episodi_id,
                    'section_index': sec['section_index'],
                    'gender': sec['gender'],
                    'minutes': sec['minutes'],
                    'eitus': sone.lemma[0],
                    'verb': lause[i+1].lemma[0],
                    'lause': " ".join(tok.text for tok in lause)
                })
            # 2) 1. ja 2. isiku pronoomenid
            if sone.lemma[0] in es_ja_te_pronoomenid:
                esimese_ja_teise_isik.append({
                    'episode_id': episodi_id,
                    'section_index': sec['section_index'],
                    'gender': sec['gender'],
                    'minutes': sec['minutes'],
                    'tunnus': sone.text.lower(),
                    'lause': " ".join(tok.text for tok in lause)
                })
            # 3) Intensiivistajad (järgneva sõna POS-i järgi)
            if sone.lemma[0] in intensiivistajad_loendist and i+1 < len(lause):
                järgnev_pos = lause[i+1].partofspeech[0]
                if järgnev_pos in ['A', 'V', 'D']:
                    intensiivistajad.append({
                        'episode_id': episodi_id, 'section_index': sec['section_index'],
                        'gender': sec['gender'], 'minutes': sec['minutes'],
                        'pos': järgnev_pos, 'tunnus': sone.lemma[0],
                        'sona': lause[i+1].lemma[0], 'lause': " ".join(tok.text for tok in lause)
                    })
                    # Omadussõnadega
                    if järgnev_pos == 'A':
                        intensiivistajad_omadussonadega.append({
                            'episode_id': episodi_id, 'section_index': sec['section_index'],
                            'gender': sec['gender'], 'minutes': sec['minutes'],
                            'pos': 'A', 'tunnus': sone.lemma[0],
                            'sona': lause[i+1].lemma[0], 'lause': " ".join(tok.text for tok in lause)
                        })
                    # Verbidega
                    elif järgnev_pos == 'V':
                        intensiivistajad_verbidega.append({
                            'episode_id': episodi_id, 'section_index': sec['section_index'],
                            'gender': sec['gender'], 'minutes': sec['minutes'],
                            'pos': 'V', 'tunnus': sone.lemma[0],
                            'sona': lause[i+1].lemma[0], 'lause': " ".join(tok.text for tok in lause)
                        })
                    # Adverbidega
                    elif järgnev_pos == 'D':
                        intensiivistajad_adverbidega.append({
                            'episode_id': episodi_id, 'section_index': sec['section_index'],
                            'gender': sec['gender'], 'minutes': sec['minutes'],
                            'pos': 'D', 'tunnus': sone.lemma[0],
                            'sona': lause[i+1].lemma[0], 'lause': " ".join(tok.text for tok in lause)
                        })
            # 4) Partiklid (I) ja sõna 'nagu', välistame tervituse 'tere'
            if sone.partofspeech[0] == 'I' or sone.lemma[0] == 'nagu':
                if sone.lemma[0] in ['tere', 'aitäh']:
                    continue  # välista tavatervitus
                # Kontrollime, et indeksid lauses on lubitud
                if 0 <= i-2 < len(lause) and 0 <= i+2 < len(lause):
                    # Lisa ainult siis, kui murranguline kontekst
                    if lause[i-2].text != sone.text or lause[i+2].text != sone.text:
                        partiklid.append({
                            'episode_id': episodi_id, 'section_index': sec['section_index'],
                            'gender': sec['gender'], 'minutes': sec['minutes'],
                            'tunnus': sone.lemma[0],
                            'lause': " ".join(tok.text for tok in lause)
                        })
            if sone.lemma[0] == 'eks' and lause[i+1].lemma[0] == 'olema':
                partiklid.append({
                    'episode_id': episodi_id, 'section_index': sec['section_index'],
                    'gender': sec['gender'], 'minutes': sec['minutes'],
                    'tunnus': 'eks ole',
                    'lause': " ".join(tok.text for tok in lause)
                })
            if sone.lemma[0] == 'jumal':
                partiklid.append({
                    'episode_id': episodi_id, 'section_index': sec['section_index'],
                    'gender': sec['gender'], 'minutes': sec['minutes'],
                    'tunnus': sone.lemma[0],
                    'lause': " ".join(tok.text for tok in lause)
                })
            # 5) Verbide eritunnused: kõnepartiklid, kõneviisid, vormid
            if sone.partofspeech[0] == 'V':
                # Kõnepartikli markerid
                if sone.text in kp_markerid:
                    kp_markerid_loend.append({
                        'episode_id': episodi_id, 'section_index': sec['section_index'],
                        'gender': sec['gender'], 'minutes': sec['minutes'],
                        'tunnus': sone.lemma[0], 'lause': " ".join(tok.text for tok in lause)
                    })
                # Tingiv kõneviis
                if sone.form[0] in tingiv_koneviis:
                    tingiv_koneviis_loend.append({
                        'episode_id': episodi_id, 'section_index': sec['section_index'],
                        'gender': sec['gender'], 'minutes': sec['minutes'],
                        'verb': sone.text, 'lause': " ".join(tok.text for tok in lause)
                    })
                # Umbisikuline vorm
                elif sone.form[0] in umbisikuline:
                    umbisikuline_loend.append({
                        'episode_id': episodi_id, 'section_index': sec['section_index'],
                        'gender': sec['gender'], 'minutes': sec['minutes'],
                        'verb': sone.text, 'lause': " ".join(tok.text for tok in lause)
                    })
                # Käskiv kõneviis
                elif sone.form[0] in kaskiv_koneviis and lause[i-1].text.lower() not in ['ei', 'et']:
                    # Erinevus verb-partikkel vs tõeline käsuvorm
                    if sone.lemma[0] in verbid_partiklitena:
                        verbid_partiklitena_loend.append({
                            'episode_id': episodi_id, 'section_index': sec['section_index'],
                            'gender': sec['gender'], 'minutes': sec['minutes'],
                            'verb': sone.text, 'lause': " ".join(tok.text for tok in lause)
                        })
                    else:
                        kaskiv_koneviis_loend.append({
                            'episode_id': episodi_id, 'section_index': sec['section_index'],
                            'gender': sec['gender'], 'minutes': sec['minutes'],
                            'verb': sone.text, 'lause': " ".join(tok.text for tok in lause)
                        })

# Muudame loendid DataFrame'ideks ja salvestame CSV-failid kontrollimiseks
df_eitused = pd.DataFrame(eitused)
df_esimese_ja_teise_isik = pd.DataFrame(esimese_ja_teise_isik)
df_intensiivistajad_omadussonadega = pd.DataFrame(intensiivistajad_omadussonadega)
df_intensiivistajad_verbidega = pd.DataFrame(intensiivistajad_verbidega)
df_intensiivistajad_adverbidega = pd.DataFrame(intensiivistajad_adverbidega)
df_tingiv = pd.DataFrame(tingiv_koneviis_loend)
df_umbisikuline = pd.DataFrame(umbisikuline_loend)
df_kaskiv = pd.DataFrame(kaskiv_koneviis_loend)
df_kp = pd.DataFrame(kp_markerid_loend)
df_verbid_partiklitena = pd.DataFrame(verbid_partiklitena_loend)
df_partiklid = pd.DataFrame(partiklid)
df_intensiivistajad = pd.DataFrame(intensiivistajad)
df_deminutiivid = pd.DataFrame(deminutiivid)

# Salvestame iga tunnuse CSV-faili
os.makedirs('tabelid_kontrollimiseks', exist_ok=True)
df_eitused.to_csv('tabelid_kontrollimiseks/eitused.csv', index=False)
df_esimese_ja_teise_isik.to_csv('tabelid_kontrollimiseks/esimese_ja_teise_isik.csv', index=False)
df_intensiivistajad_omadussonadega.to_csv('tabelid_kontrollimiseks/intensiivistajad_omadussonadega.csv', index=False)
df_intensiivistajad_verbidega.to_csv('tabelid_kontrollimiseks/intensiivistajad_verbidega.csv', index=False)
df_intensiivistajad_adverbidega.to_csv('tabelid_kontrollimiseks/intensiivistajad_adverbidega.csv', index=False)
df_tingiv.to_csv('tabelid_kontrollimiseks/tingiv.csv', index=False)
df_umbisikuline.to_csv('tabelid_kontrollimiseks/umbisikuline.csv', index=False)
df_kaskiv.to_csv('tabelid_kontrollimiseks/kaskiv.csv', index=False)
df_kp.to_csv('tabelid_kontrollimiseks/kp_markerid.csv', index=False)
df_verbid_partiklitena.to_csv('tabelid_kontrollimiseks/vaata_oota.csv', index=False)
df_partiklid.to_csv('tabelid_kontrollimiseks/partiklid.csv', index=False)
df_intensiivistajad.to_csv('tabelid_kontrollimiseks/intensiivistajad.csv', index=False)
df_deminutiivid.to_csv('tabelid_kontrollimiseks/deminutiivid.csv', index=False)

In [26]:
df = pd.read_csv('tabelid_kontrollimiseks/vaata_oota.csv')


particles = ['no', 'noh', 'nagu', 'aga', 'siis']
particle_pat = r'(?:' + '|'.join(particles) + r')'  # (?:no|noh|nagu|aga|siis)

def classify(row):
    verb = re.escape(row['verb'])
    text = row['lause']

    # verbi rea alguses, millele järgneb koma
    start_pattern = rf'(?i)^\s*{verb}\b\s*,'
    if re.match(start_pattern, text):
        return 1

    # või
    any_pattern = (
        rf'(?i)'                    
        rf'\b'                          # sõna piir
        rf'(?:{particle_pat}\s+)?'      # partikkel enne
        rf'{verb}\b'                    # verb
        rf'(?:\s+{particle_pat})?'      # partikkel pärast
        rf'\s*,'                        # tühikud (kui on), koma
    )
    return 1 if re.search(any_pattern, text) else 0

df['mood_flag'] = df.apply(classify, axis=1)
df.to_csv('tabelid_kontrollimiseks/verbid_partiklitena_kontrollimiseks.csv', index=False)


Kui tabelid on kontrollitud!!!

In [27]:
input_path  = "tabelid_kontrollimiseks/verbid_partiklitena_kontrollitud.csv"
output_path = "tabelid_kontrollimiseks/verbid_partiklitena_kontrollitud_uus.csv"

df = pd.read_csv(input_path, sep=";")
df["mood_flag"] = pd.to_numeric(df["mood_flag"], errors="coerce")
df_clean = df[df["mood_flag"] != 0]
df_clean.to_csv(output_path, sep=";", index=False)

print(f"Saved {output_path}")


Saved tabelid_kontrollimiseks/verbid_partiklitena_kontrollitud_uus.csv


In [28]:
def aggregate_section_counts(
    input_path: str,
    output_path: str,
    sep: str = ';',
    group_cols: list = None
) -> pd.DataFrame:

    """
    Agregeerib sektsioonide loendused ja arvutab tunnusõnasageduse tunnis.

    Parameetrid:
    -------------
    input_path : str
        Sisend CSV-faili tee sektsioonide loendustega.
    output_path : str
        Väljundfaili tee, kuhu salvestatakse agregeeritud tulemused.
    sep : str, valikuline
        Veerude eraldaja CSV-failis (vaikimisi ';').
    group_cols : list, valikuline
        Veergude nimed, mille alusel grupeerida. Kui None, kasutatakse
        ['episode_id', 'section_index', 'gender', 'minutes'].

    Tagastab:
    ----------
    pd.DataFrame
        DataFrame, mis sisaldab veerge grupiviisilise grupeerimise,
        'absolut_freq' ja 'freq_per_hour'.
    """
   
    df = pd.read_csv(input_path, sep=sep)
    
   # Kui grupeerimisveerud pole antud, määrame vaikimisi
    if group_cols is None:
        group_cols = ['episode_id', 'section_index', 'gender', 'minutes']
    

    grouped = (
        df
        .groupby(group_cols, as_index=False)
        .size()
        .rename(columns={'size': 'absolut_freq'})
    )
    counts = {'M': len(all_words_M), 'F': len(all_words_F)}
    
     # Arvutame tunnusõnasageduse tunnis
    grouped['freq_per_1000_tokens'] = (grouped['absolut_freq']/ (grouped['gender'].map(counts) / 1000))

    grouped.to_csv(output_path, sep=sep, index=False)
    
    print(f"Saved {output_path}")
    return grouped

In [29]:
tasks = [
    ('tabelid_kontrollimiseks/verbid_partiklitena_kontrollitud_uus.csv',
     'verbid_partiklitena_sections.csv', ';'),
    ('tabelid_kontrollimiseks/esimese_ja_teise_isik.csv',
     'esimese_ja_teise_isik_sections.csv', ','),
    ('tabelid_kontrollimiseks/partiklid.csv',
     'partikkel_sections.csv', ','),
    ('tabelid_kontrollimiseks/kaskiv_kontrollitud.csv',
     'kaskiv_sections.csv', ';'),
    ('tabelid_kontrollimiseks/tingiv.csv',
     'tingiv_sections.csv', ','),
    ('tabelid_kontrollimiseks/umbisikuline.csv',
     'umbisikuline_sections.csv', ','),
    ('tabelid_kontrollimiseks/kp_markerid.csv',
     'kp_markerid_sections.csv', ','),
    ('tabelid_kontrollimiseks/intensiivistajad_omadussonadega.csv',
     'intensiivistajad_omadussonadega_sections.csv', ','),
    ('tabelid_kontrollimiseks/intensiivistajad_verbidega.csv',
     'intensiivistajad_verbidega_sections.csv', ','),
    ('tabelid_kontrollimiseks/intensiivistajad_adverbidega.csv',
     'intensiivistajad_adverbidega_sections.csv', ','),
    ('tabelid_kontrollimiseks/eitused.csv',
     'eitused_sections.csv', ','),
    ('tabelid_kontrollimiseks/deminutiivid.csv',
     'deminutiivid_sections.csv', ',')
]

output_dir = 'tunnused_sections_csv/' #vaata GitHubis tunnused kausta
os.makedirs(output_dir, exist_ok=True)

for input_path, output_file, sep in tasks:
    out_path = os.path.join(output_dir, output_file)
    df = aggregate_section_counts(
        input_path=input_path,
        output_path=out_path,
        sep=sep,

    )

Saved tunnused_sections_csv/verbid_partiklitena_sections.csv
Saved tunnused_sections_csv/esimese_ja_teise_isik_sections.csv
Saved tunnused_sections_csv/partikkel_sections.csv
Saved tunnused_sections_csv/kaskiv_sections.csv
Saved tunnused_sections_csv/tingiv_sections.csv
Saved tunnused_sections_csv/umbisikuline_sections.csv
Saved tunnused_sections_csv/kp_markerid_sections.csv
Saved tunnused_sections_csv/intensiivistajad_omadussonadega_sections.csv
Saved tunnused_sections_csv/intensiivistajad_verbidega_sections.csv
Saved tunnused_sections_csv/intensiivistajad_adverbidega_sections.csv
Saved tunnused_sections_csv/eitused_sections.csv
Saved tunnused_sections_csv/deminutiivid_sections.csv


In [30]:
def save_tables_per_feature(
    csv_path: str,
    column_name: str,
    out_dir: str,
    threshold: int = None,
    delimiter: str = ',',
    encoding: str = 'utf-8'
):
    """
    Loeb CSV, rühmitab väärtuse järgi ja salvestab iga väärtuse CSV-failina.

    Parameetrid:
    ----------
    csv_path : str
        Tee sisend-CSV-failile.
    column_name : str
        Veeru nimi, mille alusel rühmitada.
    out_dir : str
        Väljundkataloog, kuhu failid salvestada.
    threshold : int, valikuline
        Minimaalne ridade arv väärtuse kohta, et salvestada (vaikimisi None).
    delimiter : str, valikuline
        CSV eraldaja (vaikimisi ',').
    encoding : str, valikuline
        Faili kodeering (vaikimisi 'utf-8').
    """
    df = pd.read_csv(csv_path, sep=delimiter, encoding=encoding)

    os.makedirs(out_dir, exist_ok=True)

    for part_value, group in df.groupby(column_name):
        if threshold is not None and len(group) <= threshold:
            continue

        safe = re.sub(r'[^\w\-]+', '_', str(part_value)).strip('_')
        filename = f"{safe}.csv"

        path = os.path.join(out_dir, filename)
        group.to_csv(path, index=False)
        print(f"Salvestatud {len(group)} read → {path}")


In [31]:
def make_section_freq_tables(
    input_dir: str,
    output_dir: str,
    group_cols=('episode_id', 'section_index', 'gender', 'minutes'),
    input_sep=',',
    output_sep=';'
):
    """
    Loeb iga CSV-fail input_dir, rühmitab group_cols järgi ja arvutab sageduse.
    Salvestab uued CSV-failid output_dir.
    """

    os.makedirs(output_dir, exist_ok=True)
    pattern = os.path.join(input_dir, '*.csv')
    for in_path in glob.glob(pattern):

        df = pd.read_csv(in_path, sep=input_sep)
        
        grouped = (
            df
            .groupby(list(group_cols), as_index=False)
            .size()
            .rename(columns={'size': 'absolut_freq'})
        )
        # Arvuta freq_per_hour või säilita absoluutfreq kui pole kestust
        
        counts = {'M': len(all_words_M), 'F': len(all_words_F)}
    
        
        grouped['freq_per_1000_tokens'] = (grouped['absolut_freq']/ (grouped['gender'].map(counts) / 1000))
        
        
        base = os.path.splitext(os.path.basename(in_path))[0]
        out_name = f"{base}_sections.csv"
        out_path = os.path.join(output_dir, out_name)
        grouped.to_csv(out_path, index=False, sep=output_sep)
        print(f"Salvestatud {len(grouped)} read → {out_path}")

In [32]:
def main():
    # Kõikide kogumite standardkünnis
    threshold = 50

    # Andmekogumid: (CSV-fail, grupeerimise veerg, väljundkataloog)
    datasets = [
        (
            'tabelid_kontrollimiseks/partiklid.csv',
            'tunnus',
            'particles_per_value'   #vaata GitHubis partiklid kausta
        ),

        (
            'tabelid_kontrollimiseks/kp_markerid.csv',
            'tunnus',
            'kp_markerid_per_value' #vaata GitHubis kp-markerid kausta
        ),
        (
            'tabelid_kontrollimiseks/intensiivistajad.csv',
            'tunnus',
            'intensiivistajad_per_value'       #vaata GitHubis intensiivistajad kausta
        )
    ]

    for csv_path, column_name, out_dir in datasets:
        # 1) Salvestame tabelid väärtuste kaupa
        save_tables_per_feature(
            csv_path=csv_path,
            column_name=column_name,
            out_dir=out_dir,
            threshold=threshold
        )

        # 2) Sektsioonide sagedustabelite genereerimine
        sections_dir = Path(out_dir) / f"sections_{Path(out_dir).name}"
        make_section_freq_tables(
            input_dir=out_dir,
            output_dir=str(sections_dir)
        )

if __name__ == "__main__":
    main()

Salvestatud 499 read → particles_per_value/eks_ole.csv
Salvestatud 111 read → particles_per_value/jah.csv
Salvestatud 127 read → particles_per_value/jumal.csv
Salvestatud 59 read → particles_per_value/kurat.csv
Salvestatud 10948 read → particles_per_value/nagu.csv
Salvestatud 668 read → particles_per_value/no.csv
Salvestatud 575 read → particles_per_value/noh.csv
Salvestatud 58 read → particles_per_value/oh.csv
Salvestatud 349 read → particles_per_value/okei.csv
Salvestatud 133 read → particles_per_value/vot.csv
Salvestatud 81 read → particles_per_value/sections_particles_per_value/jah_sections.csv
Salvestatud 88 read → particles_per_value/sections_particles_per_value/jumal_sections.csv
Salvestatud 146 read → particles_per_value/sections_particles_per_value/okei_sections.csv
Salvestatud 475 read → particles_per_value/sections_particles_per_value/nagu_sections.csv
Salvestatud 71 read → particles_per_value/sections_particles_per_value/vot_sections.csv
Salvestatud 183 read → particles_per